In [23]:
import numpy as np
import pandas as pd

In [24]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [25]:
train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

In [26]:
train.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,6,1,0,0,0,0,7,0,...,0,0,0,0,0,0,2,0,0,Class_6
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,0,0,0,0,0,1,0,3,0,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,0,0,7,0,1,5,2,2,0,1,...,0,4,0,2,2,0,4,3,0,Class_8
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [28]:
import missingno as msno
# msno.matrix(df=train, figsize=(20,14), color=(0.5,0,0))

In [71]:
train_x = train.drop(['target'], axis=1)
train_y = train['target']

test_x = test.copy()

In [72]:
train_x.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,0,0,6,1,0,0,0,0,7,0,...,3,0,0,0,0,0,0,2,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,3,0,0,...,8,0,0,0,0,1,0,0,0,0
3,0,0,7,0,1,5,2,2,0,1,...,0,0,4,0,2,2,0,4,3,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
train_y.head()

0    Class_6
1    Class_6
2    Class_2
3    Class_8
4    Class_2
Name: target, dtype: object

In [74]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y)
train_y = pd.Series(train_y)

In [75]:
train_y.head()

0    5
1    5
2    1
3    7
4    1
dtype: int64

In [76]:
import lightgbm as lgb
from sklearn.metrics import log_loss

In [77]:
from sklearn.model_selection import train_test_split
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True, random_state=42, stratify=train_y) 

lgb_train = lgb.Dataset(tr_x, tr_y)
lgb_eval = lgb.Dataset(va_x, va_y)

params = {'objective': 'multiclass',
                  'random_state': 10,
                  'metric': 'multi_logloss',
             'num_class': 9}




In [78]:
# train
model = lgb.train(params,
            lgb_train,
            num_boost_round=50,
            valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval])

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061396 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3286
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 75
[LightGBM] [Info] Start training from score -3.088122
[LightGBM] [Info] Start training from score -2.102454
[LightGBM] [Info] Start training from score -2.603859
[LightGBM] [Info] Start training from score -3.749957
[LightGBM] [Info] Start training from score -4.178678
[LightGBM] [Info] Start training from score -1.350710
[LightGBM] [Info] Start training from score -2.605804
[LightGBM] [Info] Start training from score -1.351627
[LightGBM] [Info] Start training from score -2.057974
[1]	train's multi_logloss: 1.87236	valid's multi_logloss: 1.87471
[2]	train's multi_logloss: 1.85248	valid's multi_logloss: 1.8567
[3]	train's multi_logloss:

In [109]:
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

logloss: 1.7535


In [110]:
pred = model.predict(test_x)
print(len(pred))

100000


In [111]:
submission = pd.DataFrame({'id': list(range(200000,300001))})
pred = pd.DataFrame(pred,columns =["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"])
submission = pd.merge(submission,pred,left_index=True, right_index=True)
submission.to_csv('./sample_submission.csv', index=False)
print('csv output')

csv output
